In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [26]:
os.chdir(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\')
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [31]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P1_testing"
path = folder + participant

In [32]:
os.chdir(path)
users={}
for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [33]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16
sensors = ["ECG", "EDA", "RESP"]

In [34]:
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

In [35]:
onset,offset = ({}, {})
horror_markers, social_markers = [],[]
for user in data.keys():
    for timestamp, markers in zip(data[user]["Markers Timestamps"], data[user]["Markers"]):
        if "Horror" in markers[0]:
            horror_markers.append(timestamp)
        if "Social" in markers[0]:
            social_markers.append(timestamp)
        if "start" in markers:
            onset[user] = timestamp
        elif "end" in markers:
            offset[user] = timestamp

In [36]:
horror_timestamp = horror_markers[-1]
social_timestamp = social_markers[-1]
horror_timestamp,social_timestamp

(93376.03895173671, 93690.08634346622)

In [37]:
onset_index = {}
offset_index = {}

for user in data.keys():
    onset_index[user] = np.where(data[user]["Signals"]["Time"] >= onset[user])[0]
    offset_index[user] = np.where(data[user]["Signals"]["Time"] <= (offset[user]-40))[-1]
    index = np.where(onset_index[user] < offset_index[user][-1])[0][-1]
    onset_index[user] = onset_index[user][:index]

In [38]:
horror_timestamps = np.where(data[user]["Signals"]["Time"]<=horror_timestamp)
social_timestamps = np.where(data[user]["Signals"]["Time"]>=horror_timestamp)

In [39]:
horror_indexes = []
for x in onset_index[user]:
    if x in horror_timestamps[0]:
        horror_indexes.append(x)
len(horror_indexes)

27110

In [40]:
social_indexes = []
for x in onset_index[user]:
    if x in social_timestamps[0]:
        social_indexes.append(x)
len(social_indexes)

29880

In [41]:
(len(horror_indexes)+len(social_indexes)),len(onset_index[user])

(56990, 56990)

In [42]:
categories = np.array(["Horror"]*len(horror_indexes) + ["Social Negative"]*len(social_indexes))

In [30]:
horror_indexes = onset_index[user][onset_index[user] == np.where(data[user]["Signals"]["Time"]<=horror_timestamp)[0]]

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_2452\4023118445.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  horror_indexes = onset_index[user][onset_index[user] == np.where(data[user]["Signals"]["Time"]<=horror_timestamp)[0]]


In [31]:
horror_indexes

array([], shape=(0, 59336), dtype=int64)

In [10]:
EEG_timestamps=[]
for timestamp in data[user]["Signals"]["Time"][onset_index[user]]:
    EEG_timestamps.append(np.where(np.array(data[user]["EEG"]["Time"]) <= timestamp)[0][-1])

In [11]:
EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)

In [ ]:
import warnings
import csv
warnings.filterwarnings("ignore")
for users in data.keys():
    bands_dict={}
    for column in EEG_filtered[users].columns:
        print(column)
        temp_dict={}
        EEG_epoch = nk.epochs_create(
            EEG_filtered[users][column].to_frame(),
            events=EEG_timestamps,
            sampling_rate=EEG_fs,
            epochs_start=0,
            epochs_end=40)
        for key in EEG_epoch.keys():
            band_power = EEG.frequencyAnalysis(EEG_epoch[key][column], EEG_fs)
            temp_dict[key] = band_power
        bands_dict[column]=temp_dict

EEG_1
EEG_2
EEG_3
EEG_4
EEG_5
EEG_6


In [ ]:
bands_dict

In [ ]:
columns = []
for channel in bands_dict.keys():
    for band in bands_dict["EEG_1"]["1"].keys():
        columns.append(channel+"_"+band)
EEG_dataframe = pd.DataFrame(columns=columns)

In [ ]:
for i,epoch in enumerate(bands_dict["EEG_1"].keys()):
    temp_list = []
    for channel in bands_dict.keys():
        for band in bands_dict[channel][epoch].keys():
            temp_list.append(bands_dict[channel][epoch][band])
    EEG_dataframe.loc[i] = temp_list

In [ ]:
EEG_dataframe

In [ ]:
EEG_dataframe.to_csv("C://Users//Cave-admin//Desktop//Users//Rodrigo//loop-system//EEG_P1.csv",index=False,header=True)

In [ ]:
EEG_dataframe = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\EEG.csv', sep=",", index_col=False)
EEG_dataframe

In [ ]:
Signals_epochs = {}
for users in data.keys():
    Signals_epochs[users] = nk.epochs_create(
        data[users]["Signals"],
        events=onset_index[users],
        sampling_rate=Opensignals_fs,
        epochs_start=0,
        epochs_end=40
    )
len(Signals_epochs['P2_testing'].keys())

In [ ]:
# %matplotlib
import matplotlib.pyplot as plt
plt.plot(Signals_epochs["P2_testing"]["62364"]["Time"],c="r")
# plt.plot(data["P2_testing"]["Signals"]["Time"],data["P2_testing"]["Signals"]["RESP"])
# plt.axvline(x=onset["P2_testing"],c="g")
# plt.axvline(x=offset["P2_testing"],c="r")

In [ ]:
import warnings
warnings.filterwarnings("ignore")
features_signals={}
for users in Signals_epochs.keys():
    temp_dict = {}
    for key in Signals_epochs[users].keys():
        temp_dict[key] = getDataframe(Signals_epochs[users][key], Opensignals_fs, resolution)
        print(key)
    features_signals[users] = temp_dict

In [ ]:
features_signals

In [ ]:
columns = features_signals["P2_testing"]["1"].columns
dataframe = pd.DataFrame(columns=columns)
for user in features_signals.keys():
    for i,epoch in enumerate(features_signals[user].keys()):
        print(i)
        dataframe.loc[i] = features_signals[user][epoch].loc[0]

In [ ]:
dataframe

In [ ]:
dataframe.to_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\dataframe_P2.csv',index=False,header=True)

In [ ]:
EEG_baseline = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\Training Models\P2\EEGbaseline.csv', sep=";", index_col=0)
EEG = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\EEG_P2.csv', sep=",", index_col=False)
dataframe_baseline = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\Training Models\P2\baseline.csv', sep=";", index_col=0)
dataframe = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\dataframe_P2.csv', sep=",", index_col=False)

In [ ]:
EEG

In [ ]:
EEG_baseline

In [ ]:
dataframe

In [ ]:
dataframe_baseline

In [ ]:
full_EEG = pd.DataFrame(columns=EEG.columns)
for index in EEG.index:
    print(index)
    full_EEG.loc[index] = EEG.loc[index]-EEG_baseline.loc[0]

In [ ]:
full_EEG

In [ ]:
full_EEG.to_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\full_EEG.csv',index=False,header=True)

In [ ]:
full_EEG = pd.read_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\full_EEG.csv', sep=",", index_col=False)

In [ ]:
full_df = pd.DataFrame(columns=dataframe.columns)
for index in dataframe.index:
    print(index)
    full_df.loc[index] = dataframe.loc[index]-dataframe_baseline.loc[0]

In [ ]:
full_df.to_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\full_df.csv',index=False,header=True)

In [ ]:
full_EEG

In [ ]:
full_dataframe = pd.concat([full_EEG, dataframe], axis=1)
full_dataframe

In [ ]:
full_dataframe.to_csv(r'C:\Users\Cave-admin\Desktop\Users\Rodrigo\loop-system\full_dataframe.csv',index=False,header=True)

In [28]:
full_dataframe = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\full_dataframe_P1.csv', sep=",", index_col=False)
full_dataframe

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,0.000023,0.000012,0.000004,0.000061,0.000221,0.000103,0.000038,0.000296,0.000007,0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000023,0.000012,0.000004,0.000061,0.000222,0.000103,0.000038,0.000299,0.000007,0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000024,0.000012,0.000004,0.000062,0.000224,0.000104,0.000038,0.000304,0.000007,0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000024,0.000012,0.000004,0.000062,0.000225,0.000104,0.000038,0.000306,0.000007,0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000024,0.000012,0.000004,0.000063,0.000227,0.000105,0.000038,0.000309,0.000008,0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56985,0.000037,0.000058,0.000045,0.000180,0.000339,0.000716,0.000674,0.000818,0.000012,0.000021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56986,0.000037,0.000059,0.000045,0.000180,0.000340,0.000717,0.000674,0.000810,0.000012,0.000021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56987,0.000038,0.000059,0.000045,0.000181,0.000342,0.000717,0.000676,0.000800,0.000012,0.000022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56988,0.000038,0.000059,0.000044,0.000182,0.000343,0.000715,0.000680,0.000795,0.000012,0.000022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\model.pkl", "rb"))

In [43]:
from sklearn import metrics
X = np.array(full_dataframe)

X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = (model.predict(X))
print("Accuracy = " + str(metrics.accuracy_score(categories,prediction)*100)+"%")

Accuracy = 47.56974907878575%


In [48]:
import csv
f = open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\simulate_P1.csv", "w")
writer = csv.writer(f)
header = ["True Value", "Prediction"]
writer.writerow(header)
for i in range(len(categories)):
    writer.writerow([str(categories[i]), str(prediction[i])])

f.close()

In [45]:
output = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\output_P1.csv', sep=";", index_col=False)
output

,Variable,True Value,Prediction
0,Category,Horror,Horror
1,Category,Horror,Horror
2,Category,Horror,Horror
3,Category,Horror,Horror
4,Category,Horror,Horror
...,...,...,...
1564,Category,Social Negative,Horror
1565,Category,Social Negative,Horror
1566,Category,Social Negative,Horror
1567,Category,Social Negative,Horror


In [47]:
print("Simulated Accuracy = " + str(metrics.accuracy_score(categories,prediction)*100)+"%")
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")

Simulated Accuracy = 47.56974907878575%
Real Accuracy = 44.74187380497132%
